In [1]:
import tensorflow as tf
from keras.layers import Conv2D, Flatten, UpSampling2D, InputLayer
from keras.models import Sequential
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
import numpy as np
import cv2 as cv
import os

2022-06-25 23:07:16.264552: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-25 23:07:16.264574: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading images and preprocess

In [123]:
# load all images from a directory
def load_images(path):
    images = []
    x = 0
    for filename in os.listdir(path):
        if x <= 50:
            img = cv.imread(os.path.join(path, filename))
            img = cv.resize(img, (256, 256))
            if img is not None:
                images.append(img)
            x += 1
    return images

In [124]:
imgs = load_images('./input/img')

In [125]:
# convert to numpy array
imgs = np.array(imgs)

#convert a list of images to lab 
lab_images = np.array([cv.cvtColor(np.float32(1.0/255*imgs[i]), cv.COLOR_BGR2LAB) for i in range(len(imgs))])

#get the L channel
l_images = np.array([np.float32(lab_images[i][:, :, 0]) for i in range(len(lab_images))])


In [126]:
l_images.shape

(51, 256, 256)

In [127]:
#get the A and B channels
ab_images = np.array([np.float32(lab_images[i][:, :, 1:]) for i in range(len(lab_images))])
ab_images /= 128
ab_images.shape

(51, 256, 256, 2)

In [129]:
l_images = l_images.reshape(51,256,256,1)
l_images.shape

(51, 256, 256, 1)

## Model

In [110]:
# Building the neural network
def buildSimpleNN():
    model = Sequential()
    model.add(InputLayer(input_shape=(None, None, 1)))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

    model.compile(optimizer='rmsprop', loss='mse')

    return model

## Training model

In [149]:
X = l_images.astype(np.float32)
Y = ab_images.astype(np.float32)

In [152]:
# Training the neural network
def trainModel(X, Y, model):
    print('Training model...')
    model.fit(x=X, y=Y, batch_size=1, epochs=30, verbose=1)
    print('Model trained.')

In [153]:
model = buildSimpleNN()

In [154]:
trainModel(X, Y, model)

Training model...
Epoch 1/30
51/51 [==============================] - 7s 99ms/step - loss: 0.0152
Epoch 2/30
51/51 [==============================] - 5s 98ms/step - loss: 0.0121
Epoch 3/30
51/51 [==============================] - 5s 100ms/step - loss: 0.0123
Epoch 4/30
51/51 [==============================] - 6s 108ms/step - loss: 0.0120
Epoch 5/30
51/51 [==============================] - 5s 98ms/step - loss: 0.0118
Epoch 6/30
51/51 [==============================] - 5s 97ms/step - loss: 0.0118
Epoch 7/30
51/51 [==============================] - 5s 95ms/step - loss: 0.0117
Epoch 8/30
51/51 [==============================] - 5s 100ms/step - loss: 0.0118
Epoch 9/30
51/51 [==============================] - 5s 104ms/step - loss: 0.0116
Epoch 10/30
51/51 [==============================] - 5s 97ms/step - loss: 0.0117
Epoch 11/30
51/51 [==============================] - 5s 98ms/step - loss: 0.0126
Epoch 12/30
51/51 [==============================] - 5s 101ms/step - loss: 0.0117
Epoch 13/30
51

In [155]:
from datetime import datetime

# Save model
model.save('./models/model_'+datetime.now().strftime("%Y_%m_%d_%H_%M_%S")+'.h5')

## Prediction

In [156]:
# Get the luminance of the image aka the image in black and white
def getLFromLab(image_loaded):
    L_image = cv.cvtColor(np.float32(
        1.0/255 * image_loaded), cv.COLOR_RGB2LAB)[:, :, 0]
    L_image = L_image.reshape(1,256,256, 1)
    return L_image

In [157]:
# Get a image with colorized pixels
def getColorizeImage(image_name, model):
    #Get image data
    image_loaded = img_to_array(load_img(image_name))
    image_loaded = cv.resize(image_loaded, (256, 256))
    image_loaded = np.array(image_loaded, dtype=float)
    #Get luminosity/Black&White image
    L_image = getLFromLab(image_loaded)
    #Predict
    output = model.predict(L_image)
    output = output * 128
    #Transform output to image
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = L_image[0][:,:,0]
    cur[:,:,1:] = output[0]
    #Convert to BGR
    cv_result = cv.cvtColor(np.float32(cur),  cv.COLOR_LAB2BGR)
    cv_result = cv_result * 255
    cv_result = cv_result.round(0)
    result_name = './output/result_'+datetime.now().strftime("%Y_%m_%d_%H_%M_%S")+'.png'
    cv.imwrite(result_name, cv_result)


In [160]:
getColorizeImage("./input/images/guada.jpg", model)

1/1 [==============================] - 0s 39ms/step
